In [ ]:
from fastapi import FastAPI
import uvicorn
import re
import json
import subprocess
import platform
import nest_asyncio

nest_asyncio.apply()
# Create an instance of the FastAPI class
sandbox = FastAPI()

# Define a path operation decorator for the root path "/"
@sandbox.post("/sandboxes/voltscript")
async def run_code(request: dict):
    prompt = request['query']
    _input = re.sub(r'http\S+', '', prompt.replace("{", "").replace("}", "").replace("\\", "/"))
    write_cmd = ['docker', 'exec', 'dind-container', 'sh', '-c', f"printf '{_input}' > input.vss"]
    _ = subprocess.run(write_cmd, capture_output=True, text=True).stdout
    run_cmd = ['docker', 'exec', 'dind-container', 'docker', 'exec', 'voltscript-sandbox', 'VoltScript', 'input.vss']
    res = subprocess.run(run_cmd, capture_output=True, text=True).stdout
    return {'response': res} 

if __name__ == "__main__":
    uvicorn.run(sandbox, host="0.0.0.0", port=8081)

INFO:     Started server process [23163]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8081 (Press CTRL+C to quit)


Print "Hej!"

['docker', 'exec', 'dind-container', 'docker', 'exec', 'voltscript-sandbox', 'VoltScript', 'input.vss']
Hej!

INFO:     127.0.0.1:49790 - "POST /sandboxes/voltscript HTTP/1.1" 200 OK


In [ ]:
docker exec dind-container cat > input.vss << EOF Print "Hello!" EOF

In [ ]:
curl -X POST -H "Content-Type: application/json" -d '{"query": "Print \"Hej!\""}' http://127.0.0.1:8081/sandboxes/voltscript


In [ ]:
curl -sfL -o docker.tgz "https://download.docker.com/linux/static/stable/x86_64/docker-25.0.4.tgz" && \
  tar -xzf docker.tgz docker/docker --strip=1 --directory /usr/local/bin && \
  rm docker.tgz